# Convert JSON to TEXT for RAG on HATVP declarations - NOW WITH METADATA!

We expand on the text extraction by adding metadata!  
Now each text will be prefaced with the name and title of the person.  
We will also chunk with 1000-char pieces.  
EDIT: no chunking for now.

## install libs

In [ ]:
!pip install datasets

     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 542.0/542.0 kB 3.2 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 116.3/116.3 kB 10.7 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 194.1/194.1 kB 4.1 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 134.8/134.8 kB 7.8 MB/s eta 0:00:00
     ━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━━ 388.9/388.9 kB 7.0 MB/s eta 0:00:00
  Attempting uninstall: huggingface-hub
    Found existing installation: huggingface-hub 0.20.3
    Uninstalling huggingface-hub-0.20.3:
      Successfully uninstalled huggingface-hub-0.20.3


In [ ]:
from datasets import load_dataset

declarations_ds = load_dataset('the-french-artist/hatvp_declarations_xml_plus_json', split='train')

/usr/local/lib/python3.10/dist-packages/huggingface_hub/utils/_token.py:89: UserWarning: 
The secret `HF_TOKEN` does not exist in your Colab secrets.
To authenticate with the Hugging Face Hub, create a token in your settings tab (https://huggingface.co/settings/tokens), set it as secret in your Google Colab and restart your session.
You will be able to reuse this secret in all of your notebooks.
Please note that authentication is recommended but still optional to access public models or datasets.
  warnings.warn(


Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
declarations_ds

Dataset({
    features: ['xml_sha1', 'declaration_xml', 'declaration_json', 'extracted_text'],
    num_rows: 10944
})

In [ ]:
declarations_df = declarations_ds.to_pandas()

In [ ]:
json.loads(declarations_df.declaration_json.to_list()[50])

{'declaration': {'dateDepot': '06/08/2020 16:19:15',
  'uuid': 'f6ceafe7-5a92-4f2d-a9d8-566989f7a9ee',
  'origine': 'ADEL',
  'complete': 'true',
  'attachedFiles': {'attachedFiles': {'fileName': 'VUE_PDF_DU_RECEPISSE_DU_DEPOT_XML',
    'serverFileName': None,
    'base64EncodedContent': ''}},
  'declarationVersion': '20171221',
  'activConsultantDto': {'neant': 'true'},
  'activProfCinqDerniereDto': {'items': {'items': {'motif': {'id': 'CREATION',
      'label': None},
     'commentaire': 'à temps partiel depuis 2014 à 80% à compter de septembre 2020 à 60%',
     'conservee': 'false',
     'description': 'Médecin hospitalier',
     'employeur': 'Hôpitaux Universitaires de Strasbourg',
     'remuneration': {'brutNet': 'Net',
      'montant': {'montant': [{'annee': '2011', 'montant': '55000'},
        {'annee': '2012', 'montant': '58000'},
        {'annee': '2013', 'montant': '62000'},
        {'annee': '2014', 'montant': '64000'},
        {'annee': '2015', 'montant': '55000'},
        

In [ ]:
test_declaration = declarations_df.declaration_json.to_list()[0]

In [ ]:
import json

parsed_json = json.loads(test_declaration)

In [ ]:
parsed_json

In [ ]:
import json

def get_person_metadata_from_dict_declaration(input_dict_declaration):
  name = input_dict_declaration['declaration']['general']['declarant']['nom'].lower()
  surname = input_dict_declaration['declaration']['general']['declarant']['prenom'].lower()
  if input_dict_declaration['declaration']['general']['qualiteDeclarantForPDF'] is None:
    return f"Fiche de {surname} {name} \n ------------ \n"
  else:
    title = input_dict_declaration['declaration']['general']['qualiteDeclarantForPDF'].lower()
    # debug json pretty print
    # print(json.dumps(input_dict_declaration, indent=2))

    return f"Fiche de {surname} {name} - {title} \n ------------ \n"

In [ ]:
print(get_person_metadata_from_dict_declaration(parsed_json))

Fiche de damien abad - député/ain(01) 
 ------------ 



In [ ]:
# extract all non-None JSON values, set them to lower
def get_all_json_values_lower(json_data):
    values = []

    # Base case: if json_data is a single value (not a dictionary or list), add it to the values list
    if not isinstance(json_data, (dict, list)):
      if json_data is not None:
           values.append(str(json_data).lower())
    elif isinstance(json_data, dict):
        # If json_data is a dictionary, recursively call get_all_values for each value
        for value in json_data.values():
            values.extend(get_all_json_values_lower(value))
    elif isinstance(json_data, list):
        # If json_data is a list, recursively call get_all_values for each element
        for item in json_data:
            values.extend(get_all_json_values_lower(item))

    return values

In [ ]:
string_version = '\n'.join(get_all_json_values_lower(parsed_json))
print(get_person_metadata_from_dict_declaration(parsed_json)+string_version)

In [ ]:
def map_extract_text_from_json(row):
  parsed_json = json.loads(row['declaration_json'])
  text_values = get_all_json_values_lower(parsed_json)
  person_metadata = get_person_metadata_from_dict_declaration(parsed_json)
  text_value = '\n'.join(text_values)
  row['extracted_text'] = person_metadata + text_value
  return row

In [ ]:
declarations_ds = declarations_ds.map(map_extract_text_from_json, num_proc=2)

Map (num_proc=2):   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
from huggingface_hub import login
from google.colab import userdata

login(userdata.get('HF_TOKEN'))

Token has not been saved to git credential helper. Pass `add_to_git_credential=True` if you want to set the git credential as well.
Token is valid (permission: write).
Your token has been saved to /root/.cache/huggingface/token
Login successful


In [ ]:
if True:
  declarations_ds.push_to_hub("the-french-artist/hatvp_declarations_xml_plus_json")

Uploading the dataset shards:   0%|          | 0/1 [00:00<?, ?it/s]

Creating parquet from Arrow format:   0%|          | 0/11 [00:00<?, ?ba/s]

README.md:   0%|          | 0.00/542 [00:00<?, ?B/s]

## check text extraction

In [ ]:
declarations_ds = load_dataset('the-french-artist/hatvp_declarations_xml_plus_json', split='train')

Generating train split:   0%|          | 0/10944 [00:00<?, ? examples/s]

In [ ]:
declarations_df = declarations_ds.to_pandas()

In [ ]:
declarations_df

,xml_sha1,declaration_xml,declaration_json,extracted_text
0,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>11/07/2022 15:40:13</d...,"{""declaration"": {""dateDepot"": ""11/07/2022 15:4...",Fiche de damien abad - député/ain(01) \n -----...
1,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>27/11/2022 18:18:23</d...,"{""declaration"": {""dateDepot"": ""27/11/2022 18:1...",Fiche de damien abad - député/ain(01) \n -----...
2,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>19/08/2022 10:08:23</d...,"{""declaration"": {""dateDepot"": ""19/08/2022 10:0...",Fiche de caroline abadie - député/isère(38) \n...
3,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>04/10/2022 17:22:07</d...,"{""declaration"": {""dateDepot"": ""04/10/2022 17:2...",Fiche de caroline abadie - député/isère(38) \n...
4,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>20/09/2021 13:41:36</d...,"{""declaration"": {""dateDepot"": ""20/09/2021 13:4...",Fiche de joelle abadie - elu départemental/hau...
...,...,...,...,...
10939,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>15/02/2022 11:07:33</d...,"{""declaration"": {""dateDepot"": ""15/02/2022 11:0...",Fiche de nadia zourgui - maire ou adjoint muni...
10940,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>17/06/2021 22:39:18</d...,"{""declaration"": {""dateDepot"": ""17/06/2021 22:3...",Fiche de comlan hubert zoutu - membre d’epci/c...
10941,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>03/12/2020 23:48:29</d...,"{""declaration"": {""dateDepot"": ""03/12/2020 23:4...",Fiche de nicolas zuili - maire ou adjoint muni...
10942,0a0a9f2a6772942557ab5355d76af442f8f65e01,<declaration><dateDepot>25/06/2021 13:29:08</d...,"{""declaration"": {""dateDepot"": ""25/06/2021 13:2...",Fiche de nicolas zuili - maire ou adjoint muni...
